In [3]:
import pyreadstat
import pandas as pd
from pymongo import MongoClient
import json
from pprint import pprint

In [4]:
df = pd.read_spss('data.sav')
df.head()

,resp,qsamptyp,qlan,state,cregion,WEIGHT,WGT_ATL_MSA,WGT_BAL_MSA,WGT_BOS_MSA,WGT_CHI_MSA,...,partyln,ideo,hh1rec,hh3rec,qpl1,qpl1a,qpc1,fertrec,SEX,qp99
0,1000001.0,LL,English,Massachusetts,Northeast,0.438028,NaN,NaN,0.513228,NaN,...,(VOL) Other/Don't know/Refused,Conservative,Three,Three,"Yes, have cell phone",NaN,NaN,Four,Female,Yes
1,1000003.0,LL,English,Maryland,South,0.718144,NaN,NaN,NaN,NaN,...,Democrat,Moderate,Three,Two,"Yes, have cell phone",NaN,NaN,One,Male,Yes
2,1000005.0,LL,English,Ohio,Midwest,0.384246,NaN,NaN,NaN,NaN,...,NaN,Very conservative,Two,Two,"Yes, have cell phone",NaN,NaN,Three,Female,Yes
3,1000006.0,LL,English,Michigan,Midwest,0.663059,NaN,NaN,NaN,NaN,...,(VOL) Other/Don't know/Refused,Very conservative,Three,Three,"Yes, have cell phone",NaN,NaN,Two,Female,Yes
4,1000008.0,LL,English,Florida,South,1.644445,NaN,NaN,NaN,NaN,...,(VOL) Other/Don't know/Refused,Very liberal?,Five or more,Two,"No, do not","Yes, someone in household has cell phone",NaN,Three,Female,Yes


In [5]:
sorted(df)

['CHDENOM',
 'CHFAMILY',
 'CHPROTFAM',
 'CHRELTRAD',
 'DENOM',
 'FAMILY',
 'PROTFAM',
 'RELTRAD',
 'SEX',
 'SPDENOM',
 'SPFAMILY',
 'SPPROTFAM',
 'SPRELTRAD',
 'WEIGHT',
 'WGT_ATL_MSA',
 'WGT_BAL_MSA',
 'WGT_BOS_MSA',
 'WGT_CHI_MSA',
 'WGT_DAL_MSA',
 'WGT_DC_MSA',
 'WGT_DET_MSA',
 'WGT_HOU_MSA',
 'WGT_LA_MSA',
 'WGT_MIA_MSA',
 'WGT_MIN_MSA',
 'WGT_NYC_MSA',
 'WGT_PHI_MSA',
 'WGT_PHO_MSA',
 'WGT_PIT_MSA',
 'WGT_PRO_MSA',
 'WGT_RIV_MSA',
 'WGT_SDI_MSA',
 'WGT_SEA_MSA',
 'WGT_SF_MSA',
 'WGT_STL_MSA',
 'WGT_TMP_MSA',
 'agerec',
 'attend',
 'born',
 'chage1rec',
 'chageoldrec',
 'chageyngrec',
 'chborn',
 'childrenrec',
 'cohort5cat',
 'cregion',
 'educ',
 'father_birthregion',
 'fertrec',
 'hh1rec',
 'hh3rec',
 'hisp',
 'ideo',
 'income',
 'income2',
 'marital',
 'mother_birthregion',
 'party',
 'partyln',
 'qa1',
 'qa2a',
 'qa2b',
 'qb1a',
 'qb1b',
 'qb1c',
 'qb20',
 'qb21',
 'qb22',
 'qb2a',
 'qb2b',
 'qb2c',
 'qb2d',
 'qb30',
 'qb30b',
 'qb31',
 'qe1',
 'qe2',
 'qe3a',
 'qe3b',
 'qe3c',

In [6]:
new_df = df[['state', 'SEX','agerec', 'cohort5cat', 'WEIGHT', 'income', 'party', 'RELTRAD', 'ideo']]
new_df.head()

,state,SEX,agerec,cohort5cat,WEIGHT,income,party,RELTRAD,ideo
0,Massachusetts,Female,65-69,Baby Boomers (born 1946-1964),0.438028,(VOL) Don't know/Refused,Independent,Catholic,Conservative
1,Maryland,Male,35-39,Generation X (born 1965-1980),0.718144,"100 to under $150,000",Independent,"Unaffiliated (religious ""nones"")",Moderate
2,Ohio,Female,70-74,Silent Generation (born 1928-1945),0.384246,"40 to under $50,000",Republican,Catholic,Very conservative
3,Michigan,Female,55-59,Baby Boomers (born 1946-1964),0.663059,"50 to under $75,000",(VOL) Don't know/Refused,Mainline Protestant Tradition,Very conservative
4,Florida,Female,30-34,Millennials (born 1981-1996),1.644445,(VOL) Don't know/Refused,(VOL) No preference,Catholic,Very liberal?


In [7]:
cleaning = new_df.rename(columns = {"SEX":"sex", "agerec":"age_group", "cohort5cat":"generation", "WEIGHT":"weight",
                                    "RELTRAD":"religion", "PROTFAM":"prot_family"})
cleaning.head()

,state,sex,age_group,generation,weight,income,party,religion,ideo
0,Massachusetts,Female,65-69,Baby Boomers (born 1946-1964),0.438028,(VOL) Don't know/Refused,Independent,Catholic,Conservative
1,Maryland,Male,35-39,Generation X (born 1965-1980),0.718144,"100 to under $150,000",Independent,"Unaffiliated (religious ""nones"")",Moderate
2,Ohio,Female,70-74,Silent Generation (born 1928-1945),0.384246,"40 to under $50,000",Republican,Catholic,Very conservative
3,Michigan,Female,55-59,Baby Boomers (born 1946-1964),0.663059,"50 to under $75,000",(VOL) Don't know/Refused,Mainline Protestant Tradition,Very conservative
4,Florida,Female,30-34,Millennials (born 1981-1996),1.644445,(VOL) Don't know/Refused,(VOL) No preference,Catholic,Very liberal?


In [8]:
cleaning['income'] = cleaning['income'].map(lambda x: x.replace('(VOL) ', ''))
cleaning['party'] = cleaning['party'].map(lambda x: x.replace('(VOL) ', ''))
cleaning['ideo'] = cleaning['ideo'].map(lambda x: x.replace('?', ''))
cleaning['ideo'] = cleaning['ideo'].map(lambda x: x.replace(', OR', ''))
cleaning['ideo'] = cleaning['ideo'].map(lambda x: x.replace('(VOL) ', ''))
cleaning['age_group'] = cleaning['age_group'].map(lambda x: x.replace('Age ', ''))
cleaning['religion'] = cleaning['religion'].map(lambda x: x.replace(' - no information on religious identity', ''))
cleaning['religion'] = cleaning['religion'].map(lambda x: x.replace(' (religious "nones")', ''))
cleaning['generation'] = cleaning['generation'].map(lambda x: x.replace(' (born 1946-1964)', ''))
cleaning['generation'] = cleaning['generation'].map(lambda x: x.replace(' (born 1965-1980)', ''))
cleaning['generation'] = cleaning['generation'].map(lambda x: x.replace(' (born 1981-1996)', ''))
cleaning['generation'] = cleaning['generation'].map(lambda x: x.replace(' (born 1928-1945)', ''))
cleaning['generation'] = cleaning['generation'].map(lambda x: x.replace(' (born before 1928)', ''))
cleaning.head()

,state,sex,age_group,generation,weight,income,party,religion,ideo
0,Massachusetts,Female,65-69,Baby Boomers,0.438028,Don't know/Refused,Independent,Catholic,Conservative
1,Maryland,Male,35-39,Generation X,0.718144,"100 to under $150,000",Independent,Unaffiliated,Moderate
2,Ohio,Female,70-74,Silent Generation,0.384246,"40 to under $50,000",Republican,Catholic,Very conservative
3,Michigan,Female,55-59,Baby Boomers,0.663059,"50 to under $75,000",Don't know/Refused,Mainline Protestant Tradition,Very conservative
4,Florida,Female,30-34,Millennials,1.644445,Don't know/Refused,No preference,Catholic,Very liberal


In [9]:
CO_independent = cleaning.loc[cleaning['state'] == 'Colorado'].loc[cleaning['party'] == 'Independent']

,state,sex,age_group,generation,weight,income,party,religion,ideo
175,Colorado,Male,75-79,Silent Generation,0.663637,"10 to under $20,000",Independent,Orthodox Christian,Liberal
519,Colorado,Female,50-54,Baby Boomers,1.140082,"100 to under $150,000",Independent,Unaffiliated,Moderate
663,Colorado,Male,70-74,Silent Generation,1.440463,"30 to under $40,000",Independent,Other Christian,Moderate
1177,Colorado,Female,40-44,Generation X,2.680726,"75 to under $100,000",Independent,Mainline Protestant Tradition,Conservative
1260,Colorado,Male,90 or older,Greatest generation,1.447972,Don't know/Refused,Independent,Jewish,Very liberal
...,...,...,...,...,...,...,...,...,...
34227,Colorado,Female,45-49,Generation X,1.920452,"50 to under $75,000",Independent,Evangelical Protestant Tradition,Conservative
34248,Colorado,Female,30-34,Millennials,0.247907,"100 to under $150,000",Independent,Unaffiliated,Very liberal
34347,Colorado,Female,70-74,Silent Generation,0.145343,"10 to under $20,000",Independent,Buddhist,Moderate
34754,Colorado,Female,24 or younger,Millennials,1.483345,"20 to under $30,000",Independent,Unaffiliated,Moderate


In [11]:
conn = 'mongodb://localhost:27017'
client = MongoClient(conn)

In [12]:
db = client.religionDB
collection = db.items

In [13]:
records = json.loads(cleaning.to_json(orient='records'))
db.collection.remove()
db.collection.insert_many(records)

<ipython-input-13-9b5b22f3ee3a>:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  db.collection.remove()


In [14]:
listings = db.collection.find()

x = 0

for listing in listings:
    x = x + 1
    
print(x)

35071
